# Data-informed parameter synthesis for population pDTMCs, notebook 4/6

The main part of this notebook is the data generation for the case models by simulating the models.
1. [Generate the data](#one)

There are four things to be setup in this notebook:
1. `agents_quantities` / `populations` - set of agents number to be considered,
2. `model_types` - set of model types to be considered,
3. `dimension_sample_size` - number of samples per parameter,
4. `n_samples` - number of samples hence the number of simulations.

The following notebook is: `analysis`

<a name="one"></a>

## GENERATE THE DATA

In [ ]:
import numpy, os, sys, pickle

In [ ]:
## create a dimension for each parameter
default_10dim_param_space = numpy.random.random((10,5))
print(default_10dim_param_space)
for row in range(len(default_10dim_param_space)):
    #print(row)
    if row >= 4:
        #print(default_10dim_param_space[row])
        default_10dim_param_space[row]=numpy.zeros(5)
    elif row >= 2:
        default_10dim_param_space[row]=default_10dim_param_space[1]
print()
print(default_10dim_param_space)

Here we generate the data by simulating the model:

In [ ]:
from src.generate_data import generate_experiments_and_data, generate_experiments, generate_data

## two-param

In [ ]:
multiparam = False
model_types = ["synchronous_parallel_"]
n_samples = [3500,1500,100]
populations = [2,3,5,10]
dimension_sample_size = 5

In [ ]:
#p_values=[]
#for v_p in numpy.random.uniform(0.0, 1.0,dimension_sample_size):
#    p_values.append(v_p)
#q_values=[]
#for v_q in numpy.random.uniform(0.0, 1.0,dimension_sample_size):
#    q_values.append(v_q)

In [ ]:
p_values = [0.028502714675268215, 0.45223461506339047, 0.8732745414252937, 0.6855555397734584, 0.13075717833714784]
q_values = [0.5057623641293089  , 0.29577906622244676, 0.8440550299528644, 0.8108008054929994, 0.03259111103419188]

In [ ]:
default_2dim_param_space = numpy.zeros((2,5))
default_2dim_param_space[0] = p_values
default_2dim_param_space[1] = q_values
default_2dim_param_space

In [ ]:
# Experiments_two_param, Data_two_param = generate_experiments_and_data(model_types,multiparam,n_samples,populations,dimension_sample_size,default_2dim_param_space)

In [ ]:
multiparam = False
model_types = ["synchronous_parallel_"]
n_samples = [3,2]
populations = [2]
dimension_sample_size = 4

In [ ]:
Experiments_two_param, Data_two_param = generate_experiments_and_data(model_types,multiparam,n_samples,populations,dimension_sample_size,default_2dim_param_space)

This took Freya 6 days to run

As you can see this process took a while so we have pickled the data and we load it back

In [ ]:
# pickle.dump( Experiments_two_param,open(os.path.join(data_path,"Experiments_two_param.p", "wb" )))
# pickle.dump(Data_two_param,open(os.path.join(data_path,"Data_two_param.p"), "wb"))

In [ ]:
Experiments_two_param = pickle.load(open(os.path.join(data_path,"Experiments_two_param.p"), "rb" ))
Data_two_param = pickle.load(open(os.path.join(data_path,"Data_two_param.p"), "rb" ))

## multiparam

## Experiments_one_point_step_down_10dim_param_space

In [ ]:
multiparam = True
model_types = ["synchronous_parallel_"]
n_samples = [3500,1500,100]
populations = [10]
dimension_sample_size = 5

In [ ]:
one_point_step_down_10dim_param_space = numpy.zeros((10,1))
for i in range(0,4):
    one_point_step_down_10dim_param_space[i] = [0.1]
for i in range(4,10):
    one_point_step_down_10dim_param_space[i] = [0]
one_point_step_down_10dim_param_space

In [ ]:
# Experiments_one_point_step_down_10dim_param_space, Data_one_point_step_down_10dim_param_space = generate_experiments_and_data(model_types,multiparam,n_samples,populations,dimension_sample_size,one_point_step_down_10dim_param_space)

In [ ]:
#pickle.dump( Experiments_one_point_step_down_10dim_param_space,open(os.path.join(data_path,"Experiments_one_point_step_down_10dim_param_space.p"), "wb" ))
#pickle.dump( Data_one_point_step_down_10dim_param_space,open(os.path.join(data_path,"Data_one_point_step_down_10dim_param_space.p"), "wb" ))

In [ ]:
Experiments_one_point_step_down_10dim_param_space = pickle.load(open(os.path.join(data_path,"Experiments_one_point_step_down_10dim_param_space.p"), "rb" ))
Data_one_point_step_down_10dim_param_space = pickle.load(open(os.path.join(data_path,"Data_one_point_step_down_10dim_param_space.p"), "rb" ))

## linear generations TBD

In [ ]:
multiparam = True
model_types = ["synchronous_parallel_"]
n_samples = [3500,1500,100]
populations = [10]
dimension_sample_size = 5

In [ ]:
one_point_linear_10dim_param_space  = numpy.zeros((10,1))
one_point_linear_10dim_param_space[0] = [0.1]
for i in range(1,10):
    one_point_linear_10dim_param_space[i] = [0.1 + i*0.5]
one_point_linear_10dim_param_space